In [1]:
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
from scipy import stats
import matplotlib.pyplot as plt
from datetime import date
from collections import defaultdict

## import the data (1991 - 2022)

In [2]:
df_atp = pd.read_csv('tennis_atp-master/atp_matches_1991.csv')
for i in range(2,10):
    df = pd.read_csv(f'tennis_atp-master/atp_matches_199{i}.csv')
    df_atp = pd.concat([df_atp,df], ignore_index=True)
for i in range(0, 23):
    b = ''
    if i < 10:
        b = 0
    df = pd.read_csv(f'tennis_atp-master/atp_matches_20{b}{i}.csv')
    df_atp = pd.concat([df_atp,df], ignore_index=True)

In [3]:
df_atp = df_atp.sort_values(by = ['tourney_date', 'match_num'], ignore_index=True)
#delete samll tournaments, davis, olympics, retirements and walkovers
df_atp = df_atp.loc[(df_atp['draw_size']>=16) & (df_atp['round']!= 'BR') & 
                    (df_atp['tourney_name'].str.contains("Olympics") == False) &
                    (df_atp['score'].str.contains("W/O") == False) &
                    (df_atp['score'].str.contains("RET") == False) &
                    (df_atp['score'].str.contains("Played and unfinished") == False)]

In [83]:
df_atp_tr = pd.read_csv('tennis_atp-master/atp_matches_1991.csv')
for i in range(2,10):
    df = pd.read_csv(f'tennis_atp-master/atp_matches_199{i}.csv')
    df_atp_tr = pd.concat([df_atp_tr,df], ignore_index=True)
for i in range(0, 23):
    b = ''
    if i < 10:
        b = 0
    df = pd.read_csv(f'tennis_atp-master/atp_matches_20{b}{i}.csv')
    df_atp_tr = pd.concat([df_atp_tr,df], ignore_index=True)

In [84]:
df_atp_tr  = df_atp_tr .sort_values(by = ['tourney_date', 'match_num'], ignore_index=True)
#delete samll tournaments, davis, olympics, retirements and walkovers
df_atp_tr  = df_atp_tr .loc[(df_atp_tr['draw_size']>=16) & (df_atp_tr['round']!= 'BR') & 
                    (df_atp_tr['tourney_name'].str.contains("Olympics") == False) &
                    (df_atp_tr['score'].str.contains("W/O") == False) &
                    (df_atp_tr['score'].str.contains("RET") == False) &
                    (df_atp_tr['score'].str.contains("DEF") == False) &
                    (df_atp_tr['score'].str.contains("Played and unfinished") == False)]

In [85]:
df_atp_tr  = df_atp_tr.drop(['winner_seed', 'winner_entry', 'loser_seed','loser_entry'], axis=1)
df_atp_tr  = df_atp_tr.dropna()
df_atp_tr  = df_atp_tr.reset_index(drop=True)
df_atp_tr 

,tourney_id,tourney_name,surface,draw_size,tourney_level,tourney_date,match_num,winner_id,winner_name,winner_hand,...,l_1stIn,l_1stWon,l_2ndWon,l_SvGms,l_bpSaved,l_bpFaced,winner_rank,winner_rank_points,loser_rank,loser_rank_points
0,1991-301,Auckland,Hard,32,A,19910107,1,101142,Emilio Sanchez,R,...,30.0,17.0,7.0,8.0,2.0,6.0,9.0,1487.0,78.0,459.0
1,1991-301,Auckland,Hard,32,A,19910107,2,101613,Malivai Washington,R,...,30.0,22.0,6.0,8.0,7.0,11.0,94.0,371.0,220.0,114.0
2,1991-338,Sydney Outdoor,Hard,32,A,19910107,2,101196,Derrick Rostagno,R,...,48.0,23.0,7.0,8.0,6.0,11.0,49.0,604.0,25.0,899.0
3,1991-301,Auckland,Hard,32,A,19910107,3,101179,Jean Philippe Fleurian,R,...,43.0,24.0,14.0,11.0,4.0,8.0,77.0,468.0,212.0,116.0
4,1991-338,Sydney Outdoor,Hard,32,A,19910107,3,101073,Martin Jaite,R,...,26.0,20.0,16.0,10.0,2.0,4.0,36.0,747.0,85.0,427.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78747,2022-0375,Montpellier,Hard,32,A,20220131,298,122330,Alexander Bublik,R,...,45.0,27.0,7.0,9.0,8.0,11.0,35.0,1433.0,36.0,1414.0
78748,2022-0375,Montpellier,Hard,32,A,20220131,299,100644,Alexander Zverev,R,...,26.0,13.0,8.0,8.0,0.0,5.0,3.0,7780.0,83.0,793.0
78749,2022-0891,Pune,Hard,32,A,20220131,299,105311,Joao Sousa,R,...,66.0,48.0,22.0,18.0,6.0,11.0,137.0,522.0,163.0,410.0
78750,2022-0375,Montpellier,Hard,32,A,20220131,300,122330,Alexander Bublik,R,...,41.0,26.0,5.0,10.0,4.0,8.0,35.0,1433.0,3.0,7780.0


In [4]:
def get_smaller_than(data, date): # date: year month day
    I = data.loc[:, 'tourney_date'] <= int(f'{date}')
    I = np.asarray(I)
    return data.loc[I]
def get_bigger_than(data, date): # date: year month day
    I = data.loc[:, 'tourney_date'] >= int(f'{date}')
    I = np.asarray(I)
    return data.loc[I]

In [5]:
df_atp = df_atp.drop(['winner_seed', 'winner_entry', 'loser_seed','loser_entry'], axis=1)
df_atp = df_atp.dropna()
df_atp = df_atp.reset_index(drop=True)
df_atp

,tourney_id,tourney_name,surface,draw_size,tourney_level,tourney_date,match_num,winner_id,winner_name,winner_hand,...,l_1stIn,l_1stWon,l_2ndWon,l_SvGms,l_bpSaved,l_bpFaced,winner_rank,winner_rank_points,loser_rank,loser_rank_points
0,1991-301,Auckland,Hard,32,A,19910107,1,101142,Emilio Sanchez,R,...,30.0,17.0,7.0,8.0,2.0,6.0,9.0,1487.0,78.0,459.0
1,1991-301,Auckland,Hard,32,A,19910107,2,101613,Malivai Washington,R,...,30.0,22.0,6.0,8.0,7.0,11.0,94.0,371.0,220.0,114.0
2,1991-338,Sydney Outdoor,Hard,32,A,19910107,2,101196,Derrick Rostagno,R,...,48.0,23.0,7.0,8.0,6.0,11.0,49.0,604.0,25.0,899.0
3,1991-301,Auckland,Hard,32,A,19910107,3,101179,Jean Philippe Fleurian,R,...,43.0,24.0,14.0,11.0,4.0,8.0,77.0,468.0,212.0,116.0
4,1991-338,Sydney Outdoor,Hard,32,A,19910107,3,101073,Martin Jaite,R,...,26.0,20.0,16.0,10.0,2.0,4.0,36.0,747.0,85.0,427.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78765,2022-0375,Montpellier,Hard,32,A,20220131,298,122330,Alexander Bublik,R,...,45.0,27.0,7.0,9.0,8.0,11.0,35.0,1433.0,36.0,1414.0
78766,2022-0375,Montpellier,Hard,32,A,20220131,299,100644,Alexander Zverev,R,...,26.0,13.0,8.0,8.0,0.0,5.0,3.0,7780.0,83.0,793.0
78767,2022-0891,Pune,Hard,32,A,20220131,299,105311,Joao Sousa,R,...,66.0,48.0,22.0,18.0,6.0,11.0,137.0,522.0,163.0,410.0
78768,2022-0375,Montpellier,Hard,32,A,20220131,300,122330,Alexander Bublik,R,...,41.0,26.0,5.0,10.0,4.0,8.0,35.0,1433.0,3.0,7780.0


In [6]:
df_2021 = get_bigger_than(df_atp, 20201231)
df_2021 = df_2021.reset_index(drop=True)

In [7]:
# fix the dataset
df_tennis = df_atp

In [8]:
# Balanced dataset
# indexes of won matches player0 -> I_0 and player -> I_1
def split_target(data):
    I_0 = np.sort(data.sample(frac=0.50).index)
    I_1 = np.arange(data.index[0], data.index[-1] + 1)
    I_1 = np.delete(I_1, I_0) 
    return I_0, I_1

In [9]:
I_0,I_1 = split_target(df_tennis)

In [10]:
def generate_data_on_Nan(column):
    m = df_tennis[column].mean()
    std = df_tennis[column].std()
    print(m, std)
    for ind in df_tennis.index:
        if np.isnan(df_tennis.at[ind, column]):
            df_tennis.at[ind, column] = np.random.normal(m, std)

We will deal with Nans only then, when we are conducting some function on them. When Nan is for example in height, we will take care of it later.

update: We delete Nans, because there is very few rows containing NaN in our dataset

# Extracting features

#### abbreviation
Player_features: (sum = 180)

    straightforward: 2 + 2 + 2 + 2 + 2 = 10
    wins&losses: 2 x 2 x 3 x 3 = 36
                 2 x 2 x 2 x 3 = 24
                 2 x 2 = 4, 2 x 2 = 4
    number of matches: 2 x 1 = 2
    wins&losses sets/games: 2 x 2 x 3 x 3 = 36
                            2 x 2 x 3 x 3 = 36
    activity: 2 x 3 = 6
              2 x 1 = 2
    elo: 2 x 1 = 2
    points_statistics: 2 x 3 x 3 = 18

Player_percents: (sum = 72)

    3 x 3 x 2 = 18
    3 x 3 x 2 = 18
    3 x 3 x 2 = 18
    2 x 3 x 2 = 12
    2 x 2 = 4
    2 x 1 = 2

Match_features: (sum = 6)

    1 + 1 + 1 + 1 + 1 + 1 = 6

Summing up: 258 + 1 (target column)

# Player features

In [11]:
player0_features = dict()
player1_features = dict()

#### straightforward data
$$\{Player_1, Player_2\} \times \{age\}$$
$$\{Player_1, Player_2\} \times \{height\}$$
$$\{Player_1, Player_2\} \times \{hand\}$$
$$\{Player_1, Player_2\} \times \{rank\}$$
$$\{Player_1, Player_2\} \times \{rank\_points\}$$

In [12]:
def check_target(ind, ind_boost, player0_features, player1_features):
    if ind_boost < len(I_1) and ind == I_0[ind_boost]:
        ind_boost += 1
        winner = player1_features
        loser = player0_features
    else:
        winner = player0_features
        loser = player1_features 
    return ind_boost, winner,loser

In [13]:
#adding features
for feature in ['age', 'ht', 'hand', 'rank', 'rank_points']:
    player0_features[feature] = []
    player1_features[feature] = []

target = []
ind_boost = 0 # boost chechikng which player was assigned to winner
for ind in tqdm(df_tennis.index): 
    ind_boost_b = ind_boost
    ind_boost,winner,loser = check_target(ind, ind_boost, player0_features, player1_features)   
    
    if ind_boost > ind_boost_b:
        target.append(1)
    else:
        target.append(0)
    for feature in ['age', 'ht', 'hand', 'rank', 'rank_points']:
        if feature == 'hand':
            winner[feature].append(int(df_tennis.at[ind, f'winner_{feature}'] == 'R'))
            loser[feature].append(int(df_tennis.at[ind, f'loser_{feature}'] == 'R'))
        else:
            winner[feature].append(df_tennis.at[ind, f'winner_{feature}'])
            loser[feature].append(df_tennis.at[ind, f'loser_{feature}'])
# Y[i] is result of X[i] match
Y = pd.DataFrame(target, columns=['target'])

#### wins in semester, year and career
$$\{Player_1\}\times\{Player_2\}\times\{wins,losses\}\times\{level_1,
level_2,level_3\}\times\{semester,year,career\}$$
level1,level2,level3 - macthes types: level1: {F,SF,QF}slam ,  {F,SF} 1000, {F}500
level2: {R16, R32} slam, {QF, R16} 1000, {SF, QF}500 
level3: {R64, R128} slam, {R32, R64} 1000, {R16, R32}500

In [14]:
# get period of time
def before_date_match(data, date, match_num): 
    I = data.loc[:, 'tourney_date'] <= int(date)
    
    I_delete = data.loc[:, 'tourney_date'] == int(date)
    I_delete &= data.loc[:, 'match_num'] >= match_num
    I =  I^I_delete
    I = np.asarray(I)
    return data.loc[I]

def semester_back(date):
    year = date // 10000
    month = (date // 100) % 100
    day = date % 100
    if month < 7:
        year -= 1
    month = (month - 6) % 12 #(-1 % 12 = 11)
    if month == 0:
        month = 12
    ans = ("%04d%02d%02d" % (year,month,day))
    return int(ans)
#get specific match (tournament_type and round_type)
def get_round_types(data, types, draw):
    I = data.loc[:, 'draw_size'] == draw
    I_t = data.loc[:, 'round'] == types[0]
    for t in types[1:]:
        I_t |= data.loc[:, 'round'] == t
    return I & I_t

def get_level(data, level):
    if level == 'level1':
        I_128 = get_round_types(data, ['F','SF','QF'], 128)
        I_64 = get_round_types(data, ['F','SF'], 64)
        I_32 = get_round_types(data, ['F'], 32) | get_round_types(data, ['F'], 16)
        I = I_128 | I_64 | I_32
    if level == 'level2':
        I_128 = get_round_types(data, ['R16','R32'], 128)
        I_64 = get_round_types(data, ['QF','R16'], 64)
        I_32 = get_round_types(data, ['SF','QF'], 32) | get_round_types(data, ['SF','QF'], 16)
        I = I_128 | I_64 | I_32
    if level == 'level3':
        I_128 = get_round_types(data, ['R64','R128'], 128)
        I_64 = get_round_types(data, ['R32','R64'], 64)
        I_32 = get_round_types(data, ['R16', 'R32'], 32) | get_round_types(data, ['R16', 'R32'], 16)
        I = I_128 | I_64 | I_32
    return data.loc[I]

In [15]:
#get number of wins in period
def get_wins(data, time, date, num, player_id):
    date = int(date)
    if time == 'semester':
        new_date = semester_back(date)
        period = get_bigger_than(data, new_date)
    if time == 'year':
        period = get_bigger_than(data, date - 10000)
    if time == 'career':
        period = data
        
    period = before_date_match(period, date, num)
    period = period['winner_id'].value_counts()
    if player_id in period:
        return period[player_id]
    else:
        return 0

#get number of losses in period
def get_losses(data, time, date, num, player_id):
    date = int(date)
    if time == 'semester':
        new_date = semester_back(date)
        period = get_bigger_than(data, new_date)
    if time == 'year':
        period = get_bigger_than(data, date - 10000)
    if time == 'career':
        period = data
        
    period = before_date_match(period, date, num)
    period = period['loser_id'].value_counts()
    if player_id in period:
        return period[player_id]
    else:
        return 0


In [16]:
# adding features
for result in ['wins', 'losses']:
    for time in ['semester', 'year', 'career']:
        for level in ['level1', 'level2', 'level3']:
            player0_features[f'{result}_{time}_{level}'] = []
            player1_features[f'{result}_{time}_{level}'] = []
ind_boost = 0
for ind in tqdm(df_tennis.index): 
    ind_boost,winner,loser = check_target(ind, ind_boost, player0_features, player1_features)
    for time in ['semester', 'year', 'career']:
        for level in ['level1', 'level2', 'level3']:
            spec_data = get_level(df_tennis, level)
            winner[f'wins_{time}_{level}'].append(get_wins(spec_data, time, df_tennis.at[ind, 'tourney_date'], 
                                           df_tennis.at[ind, 'match_num'], df_tennis.at[ind, 'winner_id']))
            loser[f'wins_{time}_{level}'].append(get_wins(spec_data, time, df_tennis.at[ind, 'tourney_date'], 
                                           df_tennis.at[ind, 'match_num'], df_tennis.at[ind, 'loser_id']))
            winner[f'losses_{time}_{level}'].append(get_losses(spec_data, time, df_tennis.at[ind, 'tourney_date'], 
                                           df_tennis.at[ind, 'match_num'], df_tennis.at[ind, 'winner_id']))
            loser[f'losses_{time}_{level}'].append(get_losses(spec_data, time, df_tennis.at[ind, 'tourney_date'], 
                                           df_tennis.at[ind, 'match_num'], df_tennis.at[ind, 'loser_id']))

#### number of matches played during career
$$\{Player_1, Player_2\} \times \{matchCounter\} $$

In [17]:
player0_features[f'match_counter'] = []
player1_features[f'match_counter'] = []
ind_boost = 0
for ind in tqdm(df_tennis.index): 
    ind_boost,winner,loser = check_target(ind, ind_boost, player0_features, player1_features)
    winner_matches = get_wins(df_tennis, 'career', df_tennis.at[ind, 'tourney_date'], 
                                df_tennis.at[ind, 'match_num'], df_tennis.at[ind, 'winner_id'])
    winner_matches += get_losses(df_tennis, 'career', df_tennis.at[ind, 'tourney_date'], 
                                df_tennis.at[ind, 'match_num'], df_tennis.at[ind, 'winner_id'])
    winner[f'match_counter'].append(winner_matches)
    
    loser_matches = get_wins(df_tennis, 'career', df_tennis.at[ind, 'tourney_date'], 
                                df_tennis.at[ind, 'match_num'], df_tennis.at[ind, 'loser_id'])
    loser_matches += get_losses(df_tennis, 'career', df_tennis.at[ind, 'tourney_date'], 
                                df_tennis.at[ind, 'match_num'], df_tennis.at[ind, 'loser_id'])
    loser[f'match_counter'].append(loser_matches)


#### sets won/lost in semester, year, career
$$\{Player_1, Player_2\} \times \{winsSet, lossesSet\} \times \{semester,year,career\} \times \{level1, level2, level3\}$$

In [22]:
#process score
def process_score(score):
    games_w, games_l = 0, 0
    splited_score = score.split('-')
    if len(splited_score) > 0 and splited_score[0].isnumeric():
        games_w = int(splited_score[0])
    #else:
        #print("PARSE ERROR", score)
    if len(splited_score) > 1 and splited_score[1].isnumeric():
        games_l = int(splited_score[1])
    elif len(splited_score) > 1:
        rm_tiebreak = splited_score[1].split('(')
        if rm_tiebreak[0].isnumeric():
            games_l = int(rm_tiebreak[0])
        #else:
            #print("PARSE ERROR", score)
    return games_w, games_l

#apply to database - get won/lost sets/games in given match
def get_w_set_in_match(score):
    score = score.split(" ")
    counter = 0
    for sets in score:
        games_w,games_l = process_score(sets)
        if games_w > games_l:
            counter += 1
    return counter

def get_l_set_in_match(score):
    score = score.split(" ")
    counter = 0
    for sets in score:
        games_w,games_l = process_score(sets)
        if games_w < games_l:
            counter += 1
    return counter

#functions to get number of won/lost sets
def get_w_set(data, time, date, num, player_id):
    date = int(date)
    if time == 'semester':
        new_date = semester_back(date)
        period = get_bigger_than(data, new_date)
    if time == 'year':
        period = get_bigger_than(data, date - 10000)
    if time == 'career':
        period = data
        
    period = before_date_match(period, date, num)
    period = period.loc[period['winner_id'] == player_id]
    return sum(period['score'].apply(get_w_set_in_match))

def get_l_set(data, time, date, num, player_id):
    date = int(date)
    if time == 'semester':
        new_date = semester_back(date)
        period = get_bigger_than(data, new_date)
    if time == 'year':
        period = get_bigger_than(data, date - 10000)
    if time == 'career':
        period = data
        
    period = before_date_match(period, date, num)
    period = period.loc[period['loser_id'] == player_id]
    return sum(period['score'].apply(get_l_set_in_match))

In [21]:
for result in ['wins_set', 'losses_set']:
    for time in ['semester', 'year', 'career']:
        for level in ['level1', 'level2', 'level3']:
            player0_features[f'{result}_{time}_{level}'] = []
            player1_features[f'{result}_{time}_{level}'] = []
ind_boost = 0
for ind in tqdm(df_tennis.index): 
    ind_boost,winner,loser = check_target(ind, ind_boost, player0_features, player1_features)
    for time in ['semester', 'year', 'career']:
        for level in ['level1', 'level2', 'level3']:
            spec_data = get_level(df_tennis, level)
            winner[f'wins_set_{time}_{level}'].append(get_w_set(spec_data, time, df_tennis.at[ind, 'tourney_date'], 
                                           df_tennis.at[ind, 'match_num'], df_tennis.at[ind, 'winner_id']))
            loser[f'wins_set_{time}_{level}'].append(get_w_set(spec_data, time, df_tennis.at[ind, 'tourney_date'], 
                                           df_tennis.at[ind, 'match_num'], df_tennis.at[ind, 'loser_id']))
            winner[f'losses_set_{time}_{level}'].append(get_l_set(spec_data, time, df_tennis.at[ind, 'tourney_date'], 
                                           df_tennis.at[ind, 'match_num'], df_tennis.at[ind, 'winner_id']))
            loser[f'losses_set_{time}_{level}'].append(get_l_set(spec_data, time, df_tennis.at[ind, 'tourney_date'], 
                                           df_tennis.at[ind, 'match_num'], df_tennis.at[ind, 'loser_id']))

PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ER

PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ER

PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ER

PARSE ERROR DEF
PARSE ERROR 
PARSE ERROR 
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR 
PARSE ERROR 
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR 
PARSE ERROR 
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR 
PARSE ERROR 
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR 
PARSE ERROR 
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR 
PARSE ERROR 
PARSE ERROR DEF
PARSE ERROR 
PARSE ERROR 
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR 
PARSE ERROR 
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ER

PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR 
PARSE ERROR 
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR 
PARSE ERROR 
PARSE ERROR DEF
PARSE ERROR 
PARSE ERROR 
PARSE ERROR 
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR 
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR 
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR 
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE 

...

PARSE ERROR 
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR 
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR 
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR 
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR 
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR 
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERRO

PARSE ERROR DEF
PARSE ERROR 
PARSE ERROR 
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR 
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
P

PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR 
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR 
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR 
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR 
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR 
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE E

PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ER

PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR Def.
PARSE ERROR Def.
PARSE ERROR Def.
PARSE ERROR DEF
PARSE ERROR Def.
PARSE ERROR Def.
PARSE ERROR Def.
PARSE ERROR Def.
PARSE ERROR Def.
PARSE ERROR Def.
PARSE ERROR Def.
PARSE ERROR Def.
PARSE ERROR Def.
PARSE ERROR Def.
PARSE ERROR Def.
PARSE ERROR Def.
PARSE ERROR Def.
PARSE ERROR Def.
PARSE ERROR Def.
PARSE ERROR Def.
PARSE ERROR Def.
PARSE ERROR Def.
PARSE ERROR DEF
PARSE ERROR DEF
PARSE ERROR Def.
PARSE ERROR Def.
PARSE ERROR Def.



#### games won/lost in semester, year and career
$$\{Player_1, Player_2\} \times \{winsGame, lossesGame\} \times \{semester,year,career\} \times \{level1, level2, level3\}$$

In [23]:
def get_w_game_in_match(score):
    score = score.split(" ")
    counter = 0
    for sets in score:
        games_w,games_l = process_score(sets)
        counter += games_w
    return counter

def get_l_game_in_match(score):
    score = score.split(" ")
    counter = 0
    for sets in score:
        games_w,games_l = process_score(sets)
        counter += games_l
    return counter

#return number of won/lost sets/games in match
def get_w_game(data, time, date, num, player_id):
    date = int(date)
    if time == 'semester':
        new_date = semester_back(date)
        period = get_bigger_than(data, new_date)
    if time == 'year':
        period = get_bigger_than(data, date - 10000)
    if time == 'career':
        period = data
        
    period = before_date_match(period, date, num)
    period = period.loc[period['winner_id'] == player_id]
    return sum(period['score'].apply(get_w_game_in_match))

def get_l_game(data, time, date, num, player_id):
    date = int(date)
    if time == 'semester':
        new_date = semester_back(date)
        period = get_bigger_than(data, new_date)
    if time == 'year':
        period = get_bigger_than(data, date - 10000)
    if time == 'career':
        period = data
        
    period = before_date_match(period, date, num)
    period = period.loc[period['loser_id'] == player_id]
    return sum(period['score'].apply(get_l_game_in_match))

In [24]:
for result in ['wins_game', 'losses_game']:
    for time in ['semester', 'year', 'career']:
        for level in ['level1', 'level2', 'level3']:
            player0_features[f'{result}_{time}_{level}'] = []
            player1_features[f'{result}_{time}_{level}'] = []
ind_boost = 0
for ind in tqdm(df_tennis.index): 
    ind_boost,winner,loser = check_target(ind, ind_boost, player0_features, player1_features)
    for time in ['semester', 'year', 'career']:
        for level in ['level1', 'level2', 'level3']:
            spec_data = get_level(df_tennis, level)
            winner[f'wins_game_{time}_{level}'].append(get_w_game(spec_data, time, df_tennis.at[ind, 'tourney_date'], 
                                           df_tennis.at[ind, 'match_num'], df_tennis.at[ind, 'winner_id']))
            loser[f'wins_game_{time}_{level}'].append(get_w_game(spec_data, time, df_tennis.at[ind, 'tourney_date'], 
                                           df_tennis.at[ind, 'match_num'], df_tennis.at[ind, 'loser_id']))
            winner[f'losses_game_{time}_{level}'].append(get_l_game(spec_data, time, df_tennis.at[ind, 'tourney_date'], 
                                           df_tennis.at[ind, 'match_num'], df_tennis.at[ind, 'winner_id']))
            loser[f'losses_game_{time}_{level}'].append(get_l_game(spec_data, time, df_tennis.at[ind, 'tourney_date'], 
                                           df_tennis.at[ind, 'match_num'], df_tennis.at[ind, 'loser_id']))
            
            

#### wins on surfaces
$$\{Player_1, Player_2\} \times \{wins,losses\}\times\{semester, career\}\times\{clay,hardcourt,grass\}$$ carpet is no more used in ATP 

In [25]:
def get_surface(data, surface):
    I = data.loc[:, 'surface'] == surface
    return data.loc[I]

In [26]:
# adding features
for result in ['wins', 'losses']:
    for time in ['semester', 'career']:
        for surface in ['Hard', 'Clay', 'Grass']:
            player0_features[f'{result}_{time}_{surface}'] = []
            player1_features[f'{result}_{time}_{surface}'] = []
ind_boost = 0
for ind in tqdm(df_tennis.index): 
    ind_boost,winner,loser = check_target(ind, ind_boost, player0_features, player1_features)
    for time in ['semester', 'career']:
        for surface in ['Hard', 'Clay', 'Grass']:
            spec_data = get_surface(df_tennis, surface)
            winner[f'wins_{time}_{surface}'].append(get_wins(spec_data, time, df_tennis.at[ind, 'tourney_date'], 
                                        df_tennis.at[ind, 'match_num'], df_tennis.at[ind, 'winner_id']))
            loser[f'wins_{time}_{surface}'].append(get_wins(spec_data, time, df_tennis.at[ind, 'tourney_date'], 
                                        df_tennis.at[ind, 'match_num'], df_tennis.at[ind, 'loser_id']))
            winner[f'losses_{time}_{surface}'].append(get_losses(spec_data, time, df_tennis.at[ind, 'tourney_date'], 
                                        df_tennis.at[ind, 'match_num'], df_tennis.at[ind, 'winner_id']))
            loser[f'losses_{time}_{surface}'].append(get_losses(spec_data, time, df_tennis.at[ind, 'tourney_date'], 
                                        df_tennis.at[ind, 'match_num'], df_tennis.at[ind, 'loser_id']))

#### head-to-head statistics
$$\{Player_1, Player_2\} \times \{year,career\}\times\{wins\}$$

In [27]:
# calculate  #wins head-to-head in period
def get_hth_wins(data, time, date, num, player_id, against_id):
    date = int(date)
    if time == 'semester':
        new_date = semester_back(date)
        period = get_bigger_than(data, new_date)
    if time == 'year':
        period = get_bigger_than(data, date - 10000)
    if time == 'career':
        period = data
        
    period = before_date_match(period, date, num)
    
    groups = period.groupby(by=['winner_id', 'loser_id']).size()
    if (player_id, against_id) in groups:
        return groups[(player_id, against_id)]
    else:
        return 0

In [28]:
# adding features
for time in ['year', 'career']:
    player0_features[f'hth_wins_{time}'] = []
    player1_features[f'hth_wins_{time}'] = []
ind_boost = 0
for ind in tqdm(df_tennis.index): 
    ind_boost,winner,loser = check_target(ind, ind_boost, player0_features, player1_features)
    for time in ['year', 'career']:
        winner[f'hth_wins_{time}'].append(get_hth_wins(df_tennis, time, df_tennis.at[ind, 'tourney_date'], 
                            df_tennis.at[ind, 'match_num'], df_tennis.at[ind, 'winner_id'], df_tennis.at[ind, 'loser_id']))
        loser[f'hth_wins_{time}'].append(get_hth_wins(df_tennis, time, df_tennis.at[ind, 'tourney_date'], 
                            df_tennis.at[ind, 'match_num'], df_tennis.at[ind, 'loser_id'], df_tennis.at[ind, 'winner_id']))

#### wins in current tournament
$$\{Player_1, Player_2\}\times\{wins,losses\}$$

In [29]:
def get_tournament(data, tourney):
    I = data.loc[:, 'tourney_name'] == tourney
    return data.loc[I]   

In [30]:
# adding features
for result in ['wins', 'losses']:
    player0_features[f'tournament_{result}'] = []
    player1_features[f'tournament_{result}'] = []
ind_boost = 0
for ind in tqdm(df_tennis.index): 
    ind_boost,winner,loser = check_target(ind, ind_boost, player0_features, player1_features)
    spec_data = get_tournament(df_tennis, df_tennis.at[ind, 'tourney_name'])
    time = 'career' # num of wins during whole career
    winner[f'tournament_wins'].append(get_wins(spec_data, time, df_tennis.at[ind, 'tourney_date'], 
                df_tennis.at[ind, 'match_num'], df_tennis.at[ind, 'winner_id']))
    loser[f'tournament_wins'].append(get_wins(spec_data, time, df_tennis.at[ind, 'tourney_date'], 
                df_tennis.at[ind, 'match_num'], df_tennis.at[ind, 'loser_id']))
    winner[f'tournament_losses'].append(get_losses(spec_data, time, df_tennis.at[ind, 'tourney_date'], 
                df_tennis.at[ind, 'match_num'], df_tennis.at[ind, 'winner_id']))
    loser[f'tournament_losses'].append(get_losses(spec_data, time, df_tennis.at[ind, 'tourney_date'], 
                df_tennis.at[ind, 'match_num'], df_tennis.at[ind, 'loser_id']))

#### minutes played recently
$$\{Player_1, Player_2\}\times\{last\_match, in\_tournament, semester\}\times\{minutes\_played\}$$

In [31]:
def get_minutes(data, time, date, num, player_id, toruney_name):
    date = int(date)
    if time == 'semester':
        new_date = semester_back(date)
        period = get_bigger_than(data, new_date)
    if time == 'in_tournament':
        period = get_bigger_than(data, date)
    if time == 'last_match':
        period = get_bigger_than(data, date)
        period = before_date_match(period, date, num)
        for ind in period.index[::-1]:
            if period.at[ind, 'winner_id'] == player_id:
                return period.at[ind, 'minutes']
        
    period = before_date_match(period, date, num)
    g1 = period.groupby(['winner_id'])['minutes'].agg('sum')
    g2 = period.groupby(['loser_id'])['minutes'].agg('sum')
    ans = 0
    
    if player_id in g1:
        ans += g1[player_id]
    if player_id in g2:
        ans += g2[player_id]
        
    return ans

In [32]:
# adding features
for time in ['last_match', 'in_tournament', 'semester']:
    player0_features[f'minutes_played_{time}'] = []
    player1_features[f'minutes_played_{time}'] = []

ind_boost = 0
for ind in tqdm(df_tennis.index): 
    ind_boost,winner,loser = check_target(ind, ind_boost, player0_features, player1_features)
    for time in ['last_match', 'in_tournament', 'semester']:
        toruney_name = df_tennis.at[ind, 'tourney_name']
        winner[f'minutes_played_{time}'].append(get_minutes(df_tennis, time, df_tennis.at[ind, 'tourney_date'], 
            df_tennis.at[ind, 'match_num'], df_tennis.at[ind, 'winner_id'], toruney_name))
        loser[f'minutes_played_{time}'].append(get_minutes(df_tennis, time, df_tennis.at[ind, 'tourney_date'], 
            df_tennis.at[ind, 'match_num'], df_tennis.at[ind, 'loser_id'], toruney_name))

#### number of weeks inactive
$$\{Player_1, Player_2\}\times\{inactive\_weeks\}$$

In [33]:
def get_break_time(date1, date2):
    year1 = date1 // 10000
    month1 = (date1 // 100) % 100
    day1 = date1 % 100
    year2 = date2 // 10000
    month2 = (date2 // 100) % 100
    day2 = date2 % 100
    d1 = date(year1, month1, day1)
    d2 = date(year2, month2, day2)
    return (d1 - d2).days / 7
    

In [34]:
def get_last_activity(data, date, num, player_id):
    date = int(date)
    # cut data to 6months year
    new_date = semester_back(date)
    period = get_bigger_than(data, new_date)
    period = before_date_match(period, date, num)
    g1 = period.groupby('winner_id')['tourney_date'].agg(min)
    g2 = period.groupby('loser_id')['tourney_date'].agg(min)
    last = date
    if player_id in g1:
        last = g1[player_id]
    if player_id in g2:
        last = min(last, g2[player_id])
    return get_break_time(date, int(last))

In [35]:
# adding features
player0_features[f'inactive_weeks'] = []
player1_features[f'inactive_weeks'] = []

ind_boost = 0
for ind in tqdm(df_tennis.index): 
    ind_boost,winner,loser = check_target(ind, ind_boost, player0_features, player1_features)
    winner[f'inactive_weeks'].append(get_last_activity(df_tennis, df_tennis.at[ind, 'tourney_date'], 
        df_tennis.at[ind, 'match_num'], df_tennis.at[ind, 'winner_id']))
    loser[f'inactive_weeks'].append(get_last_activity(df_tennis, df_tennis.at[ind, 'tourney_date'], 
        df_tennis.at[ind, 'match_num'], df_tennis.at[ind, 'loser_id']))

#### Serve and return stats
$$\{Player_1, Player_2\} \times \{PercentServeWins, PercentServeIn, PercentReturnWins\} \times \{semester,year,career\}$$

In [38]:
def get_percent_svpt(data, time, date, num, player_id):
    date = int(date)
    if time == 'semester':
        new_date = semester_back(date)
        period = get_bigger_than(data, new_date)
    if time == 'year':
        period = get_bigger_than(data, date - 10000)
    if time == 'career':
        period = data
        
    period = before_date_match(period, date, num)
    period_1 = period.loc[period['winner_id']==player_id]
    period_2 = period.loc[period['loser_id']==player_id]
    num = 0
    den = 0
    if not period_1.empty:
        num += sum(period_1['w_1stWon']) + sum(period_1['w_2ndWon'])
        den += sum(period_1['w_svpt'])
    if not period_2.empty:
        num += sum(period_2['l_1stWon']) + sum(period_2['l_2ndWon'])
        den += sum(period_2['l_svpt'])
    if den == 0:
        return 0
    
    return num / den

def get_percent_rtpt(data, time, date, num, player_id):
    date = int(date)
    if time == 'semester':
        new_date = semester_back(date)
        period = get_bigger_than(data, new_date)
    if time == 'year':
        period = get_bigger_than(data, date - 10000)
    if time == 'career':
        period = data
        
    period = before_date_match(period, date, num)
    period_1 = period.loc[period['winner_id']==player_id]
    period_2 = period.loc[period['loser_id']==player_id]
    num = 0
    den = 0
    if not period_1.empty:
        num += sum(period_1['l_svpt']) - sum(period_1['l_1stWon']) - sum(period_1['l_2ndWon'])
        den += sum(period_1['l_svpt'])
    if not period_2.empty:
        num += sum(period_2['w_svpt']) - sum(period_2['w_1stWon']) - sum(period_2['w_2ndWon'])
        den += sum(period_2['w_svpt'])
    if den == 0:
        return 0
    
    return num / den
    
def get_percent_insv(data, time, date, num, player_id):
    date = int(date)
    if time == 'semester':
        new_date = semester_back(date)
        period = get_bigger_than(data, new_date)
    if time == 'year':
        period = get_bigger_than(data, date - 10000)
    if time == 'career':
        period = data
        
    period = before_date_match(period, date, num)
    period_1 = period.loc[period['winner_id']==player_id]
    period_2 = period.loc[period['loser_id']==player_id]
    num = 0
    den = 0
    if not period_1.empty:
        num += sum(period_1['w_1stIn'])
        den += sum(period_1['w_svpt'])
    if not period_2.empty:
        num += sum(period_2['l_1stIn'])
        den += sum(period_2['l_svpt'])
    if den == 0:
        return 0
    
    return num / den

In [39]:
for time in ['semester', 'year', 'career']:
        player0_features[f'percent_svpt_{time}'] = []
        player1_features[f'percent_svpt_{time}'] = []
for time in ['semester', 'year', 'career']:
        player0_features[f'percent_rtpt_{time}'] = []
        player1_features[f'percent_rtpt_{time}'] = []
for time in ['semester', 'year', 'career']:
        player0_features[f'percent_insv_{time}'] = []
        player1_features[f'percent_insv_{time}'] = []
        
ind_boost = 0

for ind in tqdm(df_tennis.index): 
    ind_boost,winner,loser = check_target(ind, ind_boost, player0_features, player1_features)
    for time in ['semester', 'year', 'career']:
        winner[f'percent_svpt_{time}'].append(get_percent_svpt(df_tennis, time, df_tennis.at[ind, 'tourney_date'], 
                                           df_tennis.at[ind, 'match_num'], df_tennis.at[ind, 'winner_id']))
        loser[f'percent_svpt_{time}'].append(get_percent_svpt(df_tennis, time, df_tennis.at[ind, 'tourney_date'], 
                                           df_tennis.at[ind, 'match_num'], df_tennis.at[ind, 'loser_id']))
        winner[f'percent_rtpt_{time}'].append(get_percent_rtpt(df_tennis, time, df_tennis.at[ind, 'tourney_date'], 
                                           df_tennis.at[ind, 'match_num'], df_tennis.at[ind, 'winner_id']))
        loser[f'percent_rtpt_{time}'].append(get_percent_rtpt(df_tennis, time, df_tennis.at[ind, 'tourney_date'], 
                                           df_tennis.at[ind, 'match_num'], df_tennis.at[ind, 'loser_id']))
        winner[f'percent_insv_{time}'].append(get_percent_insv(df_tennis, time, df_tennis.at[ind, 'tourney_date'], 
                                           df_tennis.at[ind, 'match_num'], df_tennis.at[ind, 'winner_id']))
        loser[f'percent_insv_{time}'].append(get_percent_insv(df_tennis, time, df_tennis.at[ind, 'tourney_date'], 
                                           df_tennis.at[ind, 'match_num'], df_tennis.at[ind, 'loser_id']))

In [40]:
# create dataframe
keys = player0_features.keys()
p0_d_f = dict()
for feature in list(keys):  
    p0_d_f[f'{feature}_0'] = player0_features[f'{feature}']
        
keys = player1_features.keys()
p1_d_f = dict()
for feature in list(keys):
    p1_d_f[f'{feature}_1'] = player1_features[f'{feature}']
        
player0_df = pd.DataFrame(p0_d_f)
player1_df = pd.DataFrame(p1_d_f)

In [41]:
player_df = pd.concat([player0_df, player1_df],axis=1)
player_df

,age_0,ht_0,hand_0,rank_0,rank_points_0,wins_semester_level1_0,wins_semester_level2_0,wins_semester_level3_0,wins_year_level1_0,wins_year_level2_0,...,inactive_weeks_1,percent_svpt_semester_1,percent_svpt_year_1,percent_svpt_career_1,percent_rtpt_semester_1,percent_rtpt_year_1,percent_rtpt_career_1,percent_insv_semester_1,percent_insv_year_1,percent_insv_career_1
0,20.643395,175.0,1,78.0,459.0,0,0,0,0,0,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,21.549624,180.0,1,94.0,371.0,0,0,0,0,0,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,25.201916,185.0,1,49.0,604.0,0,0,0,0,0,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,25.322382,185.0,1,77.0,468.0,0,0,0,0,0,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,20.895277,180.0,1,85.0,427.0,0,0,0,0,0,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78765,24.624230,196.0,1,35.0,1433.0,0,1,6,0,6,...,24.0,0.625421,0.626305,0.617846,0.354866,0.358497,0.374196,0.628788,0.624565,0.640060
78766,24.783025,198.0,1,3.0,7780.0,4,11,7,8,20,...,23.0,0.599329,0.581619,0.582405,0.395741,0.394308,0.392912,0.643755,0.677505,0.671678
78767,25.809719,183.0,1,163.0,410.0,0,1,2,0,1,...,17.0,0.581081,0.574163,0.612106,0.332090,0.344828,0.362450,0.618243,0.640191,0.614130
78768,24.624230,196.0,1,35.0,1433.0,0,2,6,0,7,...,24.0,0.705628,0.687500,0.657158,0.388193,0.400116,0.387412,0.708514,0.682381,0.648206


# Player percent of wins/losses

#### percentage of wins
for every feature with wins and losses, we will take percent of it

In [42]:
wins_percent0 = dict()
wins_percent1 = dict()

In [43]:
for time in ['semester', 'year', 'career']:
    for level in ['level1', 'level2', 'level3']:
        wins_percent0[f'percent_{time}_{level}_0'] = []
        wins_percent1[f'percent_{time}_{level}_1'] = []

for time in ['semester', 'year', 'career']:
    for level in ['level1', 'level2', 'level3']:
        wins_percent0[f'percent_set_{time}_{level}_0'] = []
        wins_percent1[f'percent_set_{time}_{level}_1'] = []
        
for time in ['semester', 'year', 'career']:
    for level in ['level1', 'level2', 'level3']:
        wins_percent0[f'percent_game_{time}_{level}_0'] = []
        wins_percent1[f'percent_game_{time}_{level}_1'] = []

for time in ['semester', 'career']:
    for surface in ['Hard', 'Clay', 'Grass']:
        wins_percent0[f'percent_{time}_{surface}_0'] = []
        wins_percent1[f'percent_{time}_{surface}_1'] = []
            
for time in ['year', 'career']:
    wins_percent0[f'percent_hth_wins_{time}_0'] = []
    wins_percent1[f'percent_hth_wins_{time}_1'] = []
    

wins_percent0[f'percent_tournament'] = []
wins_percent1[f'percent_tournament'] = []

In [44]:
#adding features
for ind in tqdm(player_df.index): 
    for time in ['semester', 'year', 'career']:
        for level in ['level1', 'level2', 'level3']:
            den = (player_df.at[ind, f'wins_{time}_{level}_0'] + player_df.at[ind, f'losses_{time}_{level}_0'])
            if den != 0:
                wp0 = player_df.at[ind, f'wins_{time}_{level}_0'] / den
            else:
                wp0 = 0
            wins_percent0[f'percent_{time}_{level}_0'].append(wp0)
            
            den = (player_df.at[ind, f'wins_{time}_{level}_1'] + player_df.at[ind, f'losses_{time}_{level}_1'])
            if den != 0:
                wp1 = player_df.at[ind, f'wins_{time}_{level}_1'] / den
            else:
                wp1 = 0
            wins_percent1[f'percent_{time}_{level}_1'].append(wp1)
            
    for time in ['semester', 'year', 'career']:
        for level in ['level1', 'level2', 'level3']:
            den = (player_df.at[ind, f'wins_set_{time}_{level}_0'] + player_df.at[ind, f'losses_set_{time}_{level}_0'])
            if den != 0:
                wp0 = player_df.at[ind, f'wins_set_{time}_{level}_0'] / den
            else:
                wp0 = 0
            wins_percent0[f'percent_set_{time}_{level}_0'].append(wp0)
            
            den = (player_df.at[ind, f'wins_set_{time}_{level}_1'] + player_df.at[ind, f'losses_set_{time}_{level}_1'])
            if den != 0:
                wp1 = player_df.at[ind, f'wins_set_{time}_{level}_1'] / den
            else:
                wp1 = 0
            wins_percent1[f'percent_set_{time}_{level}_1'].append(wp1)
            
    for time in ['semester', 'year', 'career']:
        for level in ['level1', 'level2', 'level3']:
            den = (player_df.at[ind, f'wins_game_{time}_{level}_0'] + player_df.at[ind, f'losses_game_{time}_{level}_0'])
            if den != 0:
                wp0 = player_df.at[ind, f'wins_game_{time}_{level}_0'] / den
            else:
                wp0 = 0
            wins_percent0[f'percent_game_{time}_{level}_0'].append(wp0)
            
            den = (player_df.at[ind, f'wins_game_{time}_{level}_1'] + player_df.at[ind, f'losses_game_{time}_{level}_1'])
            if den != 0:
                wp1 = player_df.at[ind, f'wins_game_{time}_{level}_1'] / den
            else:
                wp1 = 0
            wins_percent1[f'percent_game_{time}_{level}_1'].append(wp1)

    for time in ['semester', 'career']:
        for surface in ['Hard', 'Clay', 'Grass']:
            den = (player_df.at[ind, f'wins_{time}_{surface}_0'] + player_df.at[ind, f'losses_{time}_{surface}_0'])
            if den != 0:
                wp0 = player_df.at[ind, f'wins_{time}_{surface}_0'] / den
            else:
                wp0 = 0
            wins_percent0[f'percent_{time}_{surface}_0'].append(wp0)
            
            den = (player_df.at[ind, f'wins_{time}_{surface}_1'] + player_df.at[ind, f'losses_{time}_{surface}_1'])
            if den != 0:
                wp1 = player_df.at[ind, f'wins_{time}_{surface}_1'] / den
            else:
                wp1 = 0  
            wins_percent1[f'percent_{time}_{surface}_1'].append(wp1)

    for time in ['year', 'career']:
        den = (player_df.at[ind, f'hth_wins_{time}_0'] + player_df.at[ind, f'hth_wins_{time}_1'])
        if den != 0:
            wp0 = player_df.at[ind, f'hth_wins_{time}_0'] / den
        else:
            wp0 = 0
        wins_percent0[f'percent_hth_wins_{time}_0'].append(wp0)
        if den != 0:
            wp1 = player_df.at[ind, f'hth_wins_{time}_1'] / den
        else:
            wp1= 0
        wins_percent1[f'percent_hth_wins_{time}_1'].append(wp1)
        
    den = (player_df.at[ind, f'tournament_wins_0'] + player_df.at[ind, f'tournament_losses_0'])
    if den != 0:
        wp0 = player_df.at[ind, f'tournament_wins_0'] / den
    else:
        wp0 = 0
    wins_percent0[f'percent_tournament'].append(wp0)
    
    den = (player_df.at[ind, f'tournament_wins_1'] + player_df.at[ind, f'tournament_losses_1'])
    if den != 0:
        wp1 = player_df.at[ind, f'tournament_wins_1'] / den
    else:
        wp0 = 0
    wins_percent1[f'percent_tournament'].append(wp1)

In [51]:
perc_wins_p0_df = pd.DataFrame(wins_percent0)
perc_wins_p1_df = pd.DataFrame(wins_percent1)
perc_wins_df = pd.concat([perc_wins_p0_df, perc_wins_p1_df], axis=1)
perc_wins_df

,percent_semester_level1_0,percent_semester_level2_0,percent_semester_level3_0,percent_year_level1_0,percent_year_level2_0,percent_year_level3_0,percent_career_level1_0,percent_career_level2_0,percent_career_level3_0,percent_set_semester_level1_0,...,percent_game_career_level3_1,percent_semester_Hard_1,percent_semester_Clay_1,percent_semester_Grass_1,percent_career_Hard_1,percent_career_Clay_1,percent_career_Grass_1,percent_hth_wins_year_1,percent_hth_wins_career_1,percent_tournament
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000
2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000
3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000
4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78765,0.000000,0.500000,0.400000,0.000000,0.500000,0.552632,0.000000,0.541667,0.488889,0.00,...,0.649300,0.411765,0.0,0.0,0.475000,0.446809,0.000000,0.0,0.0,0.777778
78766,0.571429,0.916667,1.000000,0.615385,0.800000,0.850000,0.611111,0.673913,0.772727,0.75,...,0.577093,0.562500,0.0,0.0,0.431818,0.411765,0.333333,0.0,0.0,0.666667
78767,0.000000,1.000000,0.400000,0.000000,1.000000,0.363636,0.000000,0.500000,0.263158,0.00,...,0.552937,0.333333,0.0,0.0,0.404624,0.415385,0.357143,0.0,1.0,1.000000
78768,0.000000,0.666667,0.400000,0.000000,0.538462,0.552632,0.000000,0.560000,0.488889,0.00,...,0.815057,0.833333,0.0,0.0,0.686364,0.715517,0.625000,0.0,0.0,0.888889


# Player ELO feature
calculate elo of player A as follows $R_A = 1500$ i 
$$R_A' = R_A + K(S_A - E_A)$$
$$E_A = \frac{1}{1 + 10^{(R_B - R_A)/400}}$$

In [45]:
def d_elo():
    return 1500
elo = defaultdict(d_elo)
elo_feature_0 = {'elo':[]}
elo_feature_1 = {'elo':[]}

In [46]:
def calculate_elo(player1_id, player2_id, score):
    K = 24    
    expected_score = 1 / (1 + 10 **((elo[player2_id] - elo[player1_id])/ 400))
    return elo[player1_id] + K * (score - expected_score)

In [47]:
ind_boost = 0
for ind in tqdm(df_tennis.index): 
    ind_boost,winner,loser = check_target(ind, ind_boost, elo_feature_0, elo_feature_1)
    winner_id = df_tennis.at[ind, 'winner_id']
    loser_id  = df_tennis.at[ind, 'loser_id']
    #add actual elo
    winner[f'elo'].append(elo[winner_id])  
    loser[f'elo'].append(elo[loser_id])
    
    #update elo
    winner_elo = calculate_elo(winner_id, loser_id, 1)
    loser_elo = calculate_elo(loser_id, winner_id, 0)
    elo[winner_id] = winner_elo
    elo[loser_id] = loser_elo

In [48]:
# create dataframe
keys = elo_feature_0.keys()
e0_d_f = dict()
for feature in list(keys):  
    e0_d_f[f'{feature}_0'] = elo_feature_0[f'{feature}']
        
keys = elo_feature_1.keys()
e1_d_f = dict()
for feature in list(keys):
    e1_d_f[f'{feature}_1'] = elo_feature_1[f'{feature}']
    
df_elo_0 = pd.DataFrame(e0_d_f)
df_elo_1 = pd.DataFrame(e1_d_f)

In [49]:
elo_df = pd.concat([df_elo_0, df_elo_1], axis = 1)
elo_df

,elo_0,elo_1
0,1500.000000,1500.000000
1,1500.000000,1500.000000
2,1500.000000,1500.000000
3,1500.000000,1500.000000
4,1500.000000,1500.000000
...,...,...
78765,1639.061191,1633.247085
78766,1932.562962,1607.921774
78767,1473.889208,1451.693264
78768,1650.860397,1935.771339


# Macth features

In [52]:
match_features = dict()

## surface: 0, 0.5, 1.0
$$\{surface\}$$
$$\{best\_of\}$$
$$\{match\_rank\}$$
$$\{year\}$$
$$\{sin\_day\}$$
$$\{cos\_day\}$$

In [53]:
surfaces = {'Hard':0, 'Clay':0.5, 'Grass':1.0, "Carpet":1.0}

In [54]:
def round_value(tournament_round): # value of round is points for win in round / points for win in tournament
    tmp_dict = {'F':1.0,'SF':0.6,'QF': 0.36, 'R16':0.18, 'R32': 0.09, 'R64': 0.0045, 'R128': 0.00225, 'RR': 0.00125, 'ER': 0.00125}
    return tmp_dict[tournament_round]

def norm_importance(draw_size,tournament_round) : # tournament is correalted with draw size
    MAX_LOG_DRAW_SIZE = 7
    log_draw_size = np.ceil(np.log2(draw_size))
    result = 1 / 2**(MAX_LOG_DRAW_SIZE - log_draw_size) * round_value(tournament_round)
    return result  

In [55]:
def year(t_date):
    return t_date // 10000
def sin_day(t_date):
    year = t_date // 10000
    month = (t_date // 100) % 100
    day = t_date % 100
    d = date(year,month,day)
    day_num = d.timetuple().tm_yday
    return np.sin(day_num * 2 * np.pi / 365)
def cos_day(t_date):
    year = t_date // 10000
    month = (t_date // 100) % 100
    day = t_date % 100
    d = date(year,month,day)
    day_num = d.timetuple().tm_yday
    return np.cos(day_num * 2 * np.pi / 365)

In [57]:
#adding match feature
for feature in ['surface', 'best_of', 'match_rank', 'year', 'sin_day', 'cos_day']:
    match_features[feature] = []

for ind in tqdm(df_tennis.index):
    for feature in ['surface', 'best_of', 'match_rank', 'date']:#,
        if feature == 'surface':
            match_features[feature].append(surfaces[df_tennis.at[ind, feature]])
        if feature == 'best_of':
            match_features[feature].append(df_tennis.at[ind, feature])
        if feature == 'match_rank':
            match_features[feature].append(norm_importance(df_tennis.at[ind, 'draw_size'], df_tennis.at[ind, 'round']))
        if feature == 'date':
            match_features['year'].append(year(df_tennis.at[ind,'tourney_date']))
            match_features['sin_day'].append(sin_day(df_tennis.at[ind,'tourney_date']))
            match_features['cos_day'].append(cos_day(df_tennis.at[ind,'tourney_date']))

In [58]:
match_df = pd.DataFrame(match_features)
match_df

,surface,best_of,match_rank,year,sin_day,cos_day
0,0.0,3,0.0225,1991,0.120208,0.992749
1,0.0,3,0.0225,1991,0.120208,0.992749
2,0.0,3,0.0225,1991,0.120208,0.992749
3,0.0,3,0.0225,1991,0.120208,0.992749
4,0.0,3,0.0225,1991,0.120208,0.992749
...,...,...,...,...,...,...
78765,0.0,3,0.1500,2022,0.508671,0.860961
78766,0.0,3,0.1500,2022,0.508671,0.860961
78767,0.0,3,0.1500,2022,0.508671,0.860961
78768,0.0,3,0.2500,2022,0.508671,0.860961


In [64]:
final_df = pd.concat([player_df, elo_df, perc_wins_df, match_df, Y], axis=1)
final_df

,age_0,ht_0,hand_0,rank_0,rank_points_0,wins_semester_level1_0,wins_semester_level2_0,wins_semester_level3_0,wins_year_level1_0,wins_year_level2_0,...,percent_hth_wins_year_1,percent_hth_wins_career_1,percent_tournament,surface,best_of,match_rank,year,sin_day,cos_day,target
0,20.643395,175.0,1,78.0,459.0,0,0,0,0,0,...,0.0,0.0,0.000000,0.0,3,0.0225,1991,0.120208,0.992749,1
1,21.549624,180.0,1,94.0,371.0,0,0,0,0,0,...,0.0,0.0,0.000000,0.0,3,0.0225,1991,0.120208,0.992749,0
2,25.201916,185.0,1,49.0,604.0,0,0,0,0,0,...,0.0,0.0,0.000000,0.0,3,0.0225,1991,0.120208,0.992749,0
3,25.322382,185.0,1,77.0,468.0,0,0,0,0,0,...,0.0,0.0,0.000000,0.0,3,0.0225,1991,0.120208,0.992749,0
4,20.895277,180.0,1,85.0,427.0,0,0,0,0,0,...,0.0,0.0,0.000000,0.0,3,0.0225,1991,0.120208,0.992749,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78765,24.624230,196.0,1,35.0,1433.0,0,1,6,0,6,...,0.0,0.0,0.777778,0.0,3,0.1500,2022,0.508671,0.860961,0
78766,24.783025,198.0,1,3.0,7780.0,4,11,7,8,20,...,0.0,0.0,0.666667,0.0,3,0.1500,2022,0.508671,0.860961,0
78767,25.809719,183.0,1,163.0,410.0,0,1,2,0,1,...,0.0,1.0,1.000000,0.0,3,0.1500,2022,0.508671,0.860961,1
78768,24.624230,196.0,1,35.0,1433.0,0,2,6,0,7,...,0.0,0.0,0.888889,0.0,3,0.2500,2022,0.508671,0.860961,0


In [65]:
final_df.to_csv('atp.csv', index=False)

In [89]:
final_df_reduced = pd.concat([player_df, elo_df, perc_wins_df, match_df, Y], axis=1)
final_df_reduced = final_df_reduced.loc[(final_df_reduced['match_counter_0'] >= 20) & (final_df_reduced['match_counter_1'] >= 20)]
final_df_reduced = final_df_reduced.reset_index(drop=True)
final_df_reduced

,age_0,ht_0,hand_0,rank_0,rank_points_0,wins_semester_level1_0,wins_semester_level2_0,wins_semester_level3_0,wins_year_level1_0,wins_year_level2_0,...,percent_hth_wins_year_1,percent_hth_wins_career_1,percent_tournament,surface,best_of,match_rank,year,sin_day,cos_day,target
0,22.392882,193.0,1,20.0,1079.0,0,2,4,0,2,...,0.0,0.0,1.000000,0.0,3,0.18,1991,0.956235,0.292600,1
1,22.469541,193.0,1,19.0,1114.0,0,2,4,0,2,...,1.0,1.0,1.000000,0.0,3,0.18,1991,0.993257,-0.115935,1
2,26.401095,188.0,1,22.0,1062.0,0,1,8,0,1,...,1.0,1.0,1.000000,0.0,3,0.18,1991,0.993257,-0.115935,1
3,25.215606,188.0,1,1.0,3786.0,2,5,6,2,5,...,0.0,0.0,1.000000,0.0,3,0.30,1991,0.993257,-0.115935,0
4,20.640657,185.0,1,8.0,1407.0,0,1,2,0,1,...,0.0,0.0,1.000000,0.0,3,0.30,1991,0.993257,-0.115935,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62146,24.624230,196.0,1,35.0,1433.0,0,1,6,0,6,...,0.0,0.0,0.777778,0.0,3,0.15,2022,0.508671,0.860961,0
62147,24.783025,198.0,1,3.0,7780.0,4,11,7,8,20,...,0.0,0.0,0.666667,0.0,3,0.15,2022,0.508671,0.860961,0
62148,25.809719,183.0,1,163.0,410.0,0,1,2,0,1,...,0.0,1.0,1.000000,0.0,3,0.15,2022,0.508671,0.860961,1
62149,24.624230,196.0,1,35.0,1433.0,0,2,6,0,7,...,0.0,0.0,0.888889,0.0,3,0.25,2022,0.508671,0.860961,0


In [90]:
final_df_reduced.to_csv('atp_reduced.csv', index=False)